In [1]:
import torch

x = torch.rand(5,3)
print(x)
torch.cuda.is_available()

tensor([[4.9390e-01, 6.2766e-01, 6.7025e-04],
        [2.4780e-01, 2.1460e-02, 6.4833e-01],
        [7.4443e-01, 5.6972e-01, 8.6933e-01],
        [9.1396e-01, 4.8494e-01, 7.8622e-01],
        [9.1376e-01, 4.0345e-01, 8.1417e-01]])


True

PyTorch has two primitives to work with data: ```torch.utils.data.DataLoader``` and ```torch.utils.data.Dataset```. ```Dataset``` stores the samples and their corresponding labels, and ```DataLoader``` wraps an iterable around the ```Dataset```.

PyTorch offers domain-specific libraries such as TorchText, TorchVision, and TorchAudio, all of which include datasets. We use the TorchVision dataset for this tutorial. The ```torchvision.datasets``` module contains ```Dataset``` objects for many real-world vision data like CIFAR, and COCO. In this tutorial, use FashionMNIST dataset. Every TorchVision ```Dataset``` includes two arguments ```transform``` and ```target_transform``` to modify the samples and labels respectively.
 

In [2]:
# https://pytorch.org/tutorials/beginner/basics/quickstart_tutorial.html

import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

# Download training data from open datasets.
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)

# Download test data from open datasets.
test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

  0%|          | 0/26421880 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/train-images-idx3-ubyte.gz to data/FashionMNIST/raw



  0%|          | 0/29515 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to data/FashionMNIST/raw



  0%|          | 0/4422102 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to data/FashionMNIST/raw



  0%|          | 0/5148 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to data/FashionMNIST/raw



We pass the ```Dataset``` as an argument to ```DataLoader```. This wraps an iterable over our dataset, and supports automatic batching, sampling, shuffling and multiprocess data loading. Here we define a batch size of 64, i.e. each element in the dataloader iterable will return a batch of 64 features and labels.

In [3]:
batch_size = 64

# Create data loaders.
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for X, y in test_dataloader:
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64]) torch.int64


To define a neural network in PyTorch, we create a class that inherits from ```nn.Module```. We define the layers of the network in the ```__init__``` function and specify how data will pass through the network in the ```forward``` function. To accelerate operations in the neural network, we move it to the GPU if available.

Buliding a NN: https://pytorch.org/tutorials/beginner/basics/buildmodel_tutorial.html

In [17]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

# Define model
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork().to(device)
print(model)

Using cuda device
NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


To use the model, we pass it the input data. This executes the model's ```forward``` function, along with some background operations. Do not call model.forward() directly!

Caling the model on the input returns a 10-dimensional tensor with raw predicted values for each class. We get teh prediction probabilities by passing it through an instance of the ```nn.Softmax``` module.

In [18]:
X = torch.rand(1, 28, 28, device=device)
logits = model(X)
pred_probab = nn.Softmax(dim=1)(logits)
y_pred = pred_probab.argmax(1)
print(f"Predicted class: {y_pred}")

Predicted class: tensor([7], device='cuda:0')


Many layers inside a neural network are parameterized. Subclassing ```nn.Module``` automatically tracks all fields defined inside your model object and makes all parameters accessible using your model's ```parameters()``` or ```named_parameters()``` methods.

In [19]:
print(f"Model structure: {model}\n\n")

for name, param in model.named_parameters():
    print(f"Layer: {name} | Size: {param.size()} | Values : {param[:2]} \n")


Model structure: NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


Layer: linear_relu_stack.0.weight | Size: torch.Size([512, 784]) | Values : tensor([[ 0.0101, -0.0010, -0.0293,  ...,  0.0233, -0.0082, -0.0221],
        [-0.0277, -0.0138,  0.0181,  ...,  0.0316,  0.0202, -0.0203]],
       device='cuda:0', grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.0.bias | Size: torch.Size([512]) | Values : tensor([ 0.0057, -0.0305], device='cuda:0', grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.2.weight | Size: torch.Size([512, 512]) | Values : tensor([[-0.0059,  0.0003,  0.0391,  ..., -0.0309, -0.0191,  0.0254],
        [ 0.0193, -0.0407, -0.0336,  ..., -0.0001,  0.0087, -0.0260]],
       device='cuda:0', grad_fn=<Sl

To train a model, we need a loss function and an optimizer.

Loss Functions: https://pytorch.org/docs/stable/nn.html#loss-functions

Optimizers: https://pytorch.org/docs/stable/optim.html

In [6]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

In [10]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Computer prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f} [{current:>5d}/{size:>5d}]")

In [8]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [11]:
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.304023 [    0/60000
loss: 2.297606 [ 6400/60000
loss: 2.279598 [12800/60000
loss: 2.273731 [19200/60000
loss: 2.257257 [25600/60000
loss: 2.232880 [32000/60000
loss: 2.243039 [38400/60000
loss: 2.206182 [44800/60000
loss: 2.205826 [51200/60000
loss: 2.182531 [57600/60000
Test Error: 
 Accuracy: 38.3%, Avg loss: 2.176574 

Epoch 2
-------------------------------
loss: 2.182367 [    0/60000
loss: 2.181051 [ 6400/60000
loss: 2.124323 [12800/60000
loss: 2.142515 [19200/60000
loss: 2.094232 [25600/60000
loss: 2.042611 [32000/60000
loss: 2.074215 [38400/60000
loss: 1.990848 [44800/60000
loss: 2.003083 [51200/60000
loss: 1.948423 [57600/60000
Test Error: 
 Accuracy: 55.4%, Avg loss: 1.938179 

Epoch 3
-------------------------------
loss: 1.962338 [    0/60000
loss: 1.944558 [ 6400/60000
loss: 1.825857 [12800/60000
loss: 1.869030 [19200/60000
loss: 1.757936 [25600/60000
loss: 1.712341 [32000/60000
loss: 1.738617 [38400/60000
loss: 1.624431 [4480

In [12]:
torch.save(model.state_dict(), "model.pth")
print("Saved PyTorch Model State to model.pth")

Saved PyTorch Model State to model.pth


In [14]:
model = NeuralNetwork()
model.load_state_dict(torch.load("model.pth"))

<All keys matched successfully>

In [15]:
classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]

model.eval()
x, y = test_data[0][0], test_data[0][1]
with torch.no_grad():
    pred = model(x)
    predicted, actual = classes[pred[0].argmax(0)], classes[y]
    print(f'Predicted: "{predicted}", Actual: "{actual}"')

Predicted: "Ankle boot", Actual: "Ankle boot"
